In [3]:
#!pip install deepspeed

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
import deepspeed

# Load model and tokenizer
model_name = "~/.cache/huggingface/hub/models--gpt2"  # Replace with your large model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Enable gradient checkpointing to reduce memory usage
model.gradient_checkpointing_enable()

# Define training arguments with deepspeed
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,  # Adjust based on memory capacity
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    save_strategy="epoch",
    fp16=True,  # Mixed precision training
    deepspeed="./deepspeed_config.json"  # Specify the DeepSpeed config file
)

# Create the DeepSpeed config
ds_config = {
    "zero_optimization": {
        "stage": 2,  # Enable model sharding
        "offload_optimizer": {"device": "cpu", "pin_memory": True},
        "offload_param": {"device": "cpu", "pin_memory": True}
    },
    "fp16": {
        "enabled": True
    }
}

# Save DeepSpeed config to a JSON file
import json
with open("deepspeed_config.json", "w") as f:
    json.dump(ds_config, f)

# Prepare training data
train_dataset = ...  # Define your dataset

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

# Start training
trainer.train()

ModuleNotFoundError: No module named 'deepspeed'